In [1]:
import json
import sys
import copy
import asyncio
from pathlib import Path

# Добавляем корневую директорию проекта в пути импорта
project_root = Path.cwd().parent
sys.path.append(str(project_root))

# Импортируем необходимые модули
from src.hh.api_client import HHApiClient
from src.tg_bot.handlers.spec_handlers.resume_update_handler import _prepare_hh_api_payload

# Пути к файлам
original_resume_path = Path("tests/fetched_resume_6d807532ff0ed6b79f0039ed1f63386d724a62.json")
updated_resume_path = Path("updated_resume_result.json")  # Путь к файлу с обновленным LLM резюме

# Проверяем наличие файлов
print(f"Оригинальное резюме существует: {original_resume_path.exists()}")
print(f"Обновленное резюме существует: {updated_resume_path.exists()}")

# Загружаем данные
with open(original_resume_path, 'r', encoding='utf-8') as f:
    original_resume_data = json.load(f)
print("Данные оригинального резюме успешно загружены")

with open(updated_resume_path, 'r', encoding='utf-8') as f:
    updated_resume_data = json.load(f)
print("Данные обновленного резюме успешно загружены")

# Получаем ID резюме из оригинальных данных
resume_id = original_resume_data.get("id")
if not resume_id:
    print("❌ Ошибка: Не удалось получить ID резюме из оригинальных данных")
else:
    print(f"ID резюме: {resume_id}")

2025-05-18 12:12:39,724 - INFO - states - <module> - Инициализация состояний пользователя
2025-05-18 12:12:39,725 - INFO - text_constants - <module> - Инициализация словарей с текстами сообщений


Оригинальное резюме существует: True
Обновленное резюме существует: True
Данные оригинального резюме успешно загружены
Данные обновленного резюме успешно загружены
ID резюме: 6d807532ff0ed6b79f0039ed1f63386d724a62


In [2]:
access_token = "USERJRKKJDHGBRNQB1VEOOI9MVVAF8Q542P7E3MP5FA3TKCJJGVCK2S48KUUP4GR"
refresh_token = "USERULA5I428ICTFPTQ5ESH330LVJ1FUGT63ICQV3U95R63IJFB6ORKNK2PVE1SC"

In [3]:
if not access_token or not refresh_token:
    print("❌ Ошибка: Не указаны токены доступа")
else:
    # Асинхронная функция для обновления резюме
    async def update_resume_on_hh():
        try:
            print("\n=== ПОДГОТОВКА ДАННЫХ ДЛЯ ОТПРАВКИ НА HH.RU ===")
            
            # Подготавливаем payload для API HH.ru
            prepared_payload = _prepare_hh_api_payload(
                original_resume_data, updated_resume_data, resume_id
            )
            
            # Вывод payload для проверки (можно закомментировать в продакшене)
            print("\nПодготовленный payload для API HH.ru:")
            payload_preview = {k: v for k, v in list(prepared_payload.items())[:5]}  # Показываем только первые 5 полей
            print(json.dumps(payload_preview, ensure_ascii=False, indent=2) + "...")
            
            print("\n=== ОТПРАВКА ЗАПРОСА НА ОБНОВЛЕНИЕ РЕЗЮМЕ ===")
            
            # Создаем клиент API
            hh_client = HHApiClient(access_token, refresh_token)
            
            # Выполняем запрос к API для обновления резюме
            api_response = await hh_client.request(
                endpoint=f'resumes/{resume_id}',
                method='PUT',
                data=prepared_payload
            )
            
            # Проверяем ответ
            if not api_response:  # Для методов, возвращающих 204 No Content
                print("\n✅ Резюме успешно обновлено на HH.ru! (Получен ответ 204 No Content)")
            else:
                print(f"\n✅ Резюме обновлено. Ответ API: {api_response}")
            
            # Для получения обновленных данных резюме
            print("\n=== ПОЛУЧЕНИЕ ОБНОВЛЕННОГО РЕЗЮМЕ С СЕРВЕРА ===")
            updated_resume_from_hh = await hh_client.request(
                endpoint=f'resumes/{resume_id}',
                method='GET'
            )
            
            if updated_resume_from_hh:
                # Сохраняем обновленное резюме с сервера для проверки
                updated_resume_from_hh_path = Path("hh_updated_resume.json")
                with open(updated_resume_from_hh_path, 'w', encoding='utf-8') as f:
                    json.dump(updated_resume_from_hh, f, ensure_ascii=False, indent=2)
                print(f"Обновленное резюме с HH.ru сохранено в файл: {updated_resume_from_hh_path}")
                
                # Выводим основные поля для быстрой проверки
                print("\nОсновные поля обновленного резюме:")
                print(f"Должность: {updated_resume_from_hh.get('title', 'Н/Д')}")
                
                skills_preview = updated_resume_from_hh.get('skills', 'Н/Д')
                if len(skills_preview) > 100:
                    skills_preview = skills_preview[:100] + "..."
                print(f"Навыки (превью): {skills_preview}")
                
                skill_set = updated_resume_from_hh.get('skill_set', [])
                print(f"Ключевые навыки ({len(skill_set)}): {', '.join(skill_set[:5])}...")
            
        except Exception as e:
            print(f"\n❌ Ошибка при обновлении резюме: {e}")

    # Запускаем асинхронную функцию
    await update_resume_on_hh()

2025-05-18 12:12:46,169 - WARNING - resume_update_handler - _prepare_hh_api_payload - LLM предложил изменения для 'professional_roles' резюме 6d807532ff0ed6b79f0039ed1f63386d724a62, но они будут проигнорированы, так как требуется поле 'id' для каждой роли.
2025-05-18 12:12:46,171 - INFO - resume_update_handler - _clean_payload - Удаление устаревшего поля 'specialization' из payload для резюме 6d807532ff0ed6b79f0039ed1f63386d724a62
2025-05-18 12:12:46,172 - WARNING - resume_update_handler - _clean_payload - Поле 'employment_form' некорректно в payload для резюме 6d807532ff0ed6b79f0039ed1f63386d724a62. Установка значения по умолчанию: {'id': 'full'}.



=== ПОДГОТОВКА ДАННЫХ ДЛЯ ОТПРАВКИ НА HH.RU ===

Подготовленный payload для API HH.ru:
{
  "last_name": "Немов",
  "first_name": "Максим",
  "middle_name": "Юрьевич",
  "title": "ML Engineer / NLP Specialist",
  "created_at": "2025-05-16T08:25:26+0300"
}...

=== ОТПРАВКА ЗАПРОСА НА ОБНОВЛЕНИЕ РЕЗЮМЕ ===


2025-05-18 12:12:48,615 - INFO - hh_api_client - request - Запрос: PUT https://api.hh.ru/resumes/6d807532ff0ed6b79f0039ed1f63386d724a62, Статус: 204



✅ Резюме успешно обновлено на HH.ru! (Получен ответ 204 No Content)

=== ПОЛУЧЕНИЕ ОБНОВЛЕННОГО РЕЗЮМЕ С СЕРВЕРА ===


2025-05-18 12:12:50,100 - INFO - hh_api_client - request - Запрос: GET https://api.hh.ru/resumes/6d807532ff0ed6b79f0039ed1f63386d724a62, Статус: 200


Обновленное резюме с HH.ru сохранено в файл: hh_updated_resume.json

Основные поля обновленного резюме:
Должность: ML Engineer / NLP Specialist
Навыки (превью): Специалист с глубокими знаниями в области NLP и LLM, включая методы fine-tuning и разработку RAG-сис...
Ключевые навыки (26): PyTorch, Hugging Face Transformers, LangChain, Chroma, Pinecone...
